In [1]:
from NewWordDiscovery import word_extraction
import matplotlib.pyplot as plt
%matplotlib inline
#reload(word_extraction)

## 前處理

去標點和英文數字、將文章分成太陽花前和太陽花後。

In [4]:
puncts = [u'，', u'。', u'、', u'"', u'.', u'~', u'!', u'?', u'！', u'？', u'=', u'-', u';',u'ˋ',u'|', u'；',u'‧', u'╴',u'(', u')',u')'u'（',u'）',u',', u'」',u'「',u'<', u'>', u':',u'：' ,u'—',u'@@@',u'《',u'》', u'【', u'】', u'/',u'═',u'＝', u'@',u'～', u'[', u']']
english = [chr(x).decode('utf8') for x in range(65,91)] + [chr(x).decode('utf8') for x in range(97,123)]
number = [u'0',u'1',u'2',u'3',u'4',u'5',u'6',u'7',u'8',u'9']
f = open('NewWordDiscovery/all_utf8.txt','r')
former_sents, latter_sents = [], []
for sent in f:
    sent = sent.strip()
    segs = sent.split('&&&')
    time = segs[1].split('-')
    sent = ' '.join([segs[0],segs[2],segs[3]])
    sent = sent.decode('utf8')
    if (time[1] == 'Mar' and int(time[2]) >= 17) or time[1] == 'Apr':
        for x in puncts+english+number:
            sent = sent.replace(x,u' ')
        latter_sents.extend(sent.split())
    
    else:
        for x in puncts+english+number:
            sent = sent.replace(x,u' ')
        former_sents.extend(sent.split())
    

## 切NGram & 計算branching entropy

我們將上面前處理好的句子做NGram(最大曾經做到8-gram，此處取5-gram做展示)處理，並且計算其branching entropy:

### Branching Entropy

Branching Entropy(BE): 用字串在文本中的上、下一個字的不確定性來評估一個字元序列是不是一個詞

一個完整的詞，它的"上一個"字和"下一個"字出現的可能性會有很多種，
如果一個字串不是詞，則它必然是詞的一部分，因此它的上或下會接的字可能性通常不多，
例如"式設" 這個字串，我們很快地會斷定它的上下兩字很可能是"程"、"計"，不確定性低，
因此"式設"這個字串很可能不是一個詞。
反之，"服貿協議"，這四個字的上下字在沒有其它資訊的狀況下不易判斷，因此它很可能是一個詞。

在資訊科學上，我們會用Entropy來量化資訊的不確定性，
Branching Entropy就是來自這裡(reference: http://www.aclweb.org/anthology/P06-2056)：
BE(gram) = -1 sum([ P(x|gram) log(P(x|gram)) for x in possible next character])
Branching Entropy愈大代表不確定性愈高，這個序列就愈可能是一個詞。

實際上，我們需要計算上一個字的branching entropy和下一個字的branching entropy，在這裡我們取兩者的maximum當作篩選指標。
在bigram和trigram的狀況下，若branching entropy < 0.9則去掉，
4-gram以上的狀況若 < 0.8則去掉，
閾值的選擇則來自觀察。
另外在計算branching entropy時也設下了詞頻門檻，bigram須大於15、trigram以上須大於10才將之納入候選字串。

In [13]:
bigrams = word_extraction.NGrams(latter_sents,2,min_count=0)
bigram_be = [(gram, word_extraction.branch_entropy(gram,bigrams.ngrams)) for gram in bigrams.ngrams[2] if bigrams.ngrams[2][gram].count > 15]
bigram_be = {x[0]:x[1] for x in bigram_be if x[1] > 0.9}
trigrams = word_extraction.NGrams(latter_sents,3,min_count=0)
trigram_be = [(gram, word_extraction.branch_entropy(gram,trigrams.ngrams)) for gram in trigrams.ngrams[3] if trigrams.ngrams[3][gram].count > 10]
trigram_be = {x[0]:x[1] for x in trigram_be if x[1] > 0.9}
fourgrams = word_extraction.NGrams(latter_sents, 4, min_count=0)
fourgram_be = [(gram, word_extraction.branch_entropy(gram,fourgrams.ngrams)) for gram in fourgrams.ngrams[4] if fourgrams.ngrams[4][gram].count > 10]
fourgram_be = {x[0]:x[1] for x in fourgram_be if x[1]>0.8}
fivegrams = word_extraction.NGrams(latter_sents, 5, min_count=0)
fivegram_be = [(gram, word_extraction.branch_entropy(gram,fivegrams.ngrams)) for gram in fivegrams.ngrams[5] if fivegrams.ngrams[5][gram].count > 10]
fivegram_be = {x[0]:x[1] for x in fivegram_be if x[1]>0.8}


## 事後篩選(1)

用branching entropy去掉不太可能是詞的字串後，再用既有的字典以及太陽花前出現的ngram篩掉舊詞。

In [4]:
fr =open('chinese_dict.txt','r')
chinese_dict = [x.strip().split()[0].decode('utf8') for x in fr]
chinese_dict = set(chinese_dict)

former_bigrams = word_extraction.NGrams(former_sents,2,min_count=0)
bigram_be = {x:y for x,y in bigram_be.items() if x not in former_bigrams.ngrams[2]}
last_bigram_be = {x:y for x,y in bigram_be.items() if x not in chinese_dict}
former_trigrams = word_extraction.NGrams(former_sents,3,min_count=0)
trigram_be = {x:y for x,y in trigram_be.items() if x not in former_trigrams.ngrams[3]}
last_trigram_be = {x:y for x,y in trigram_be.items() if x not in chinese_dict}
former_fourgrams = word_extraction.NGrams(former_sents,4,min_count=0)
fourgram_be = {x:y for x,y in fourgram_be.items() if x not in former_fourgrams.ngrams[4]}
last_fourgram_be = {x:y for x,y in fourgram_be.items() if x not in chinese_dict}
former_fivegrams = word_extraction.NGrams(former_sents,5,min_count=0)
fivegram_be = {x:y for x,y in fivegram_be.items() if x not in former_fivegrams.ngrams[5]}
last_fivegram_be = {x:y for x,y in fivegram_be.items() if x not in chinese_dict}

fw = open('possible_grams','w')
for x in last_bigram_be.keys()+last_trigram_be.keys()+last_fourgram_be.keys()+last_fivegram_be.keys():
    fw.write(x.encode('utf8')+'\n')
fw.close()

## 事後篩選(2)

再用另一個指標: Symmetrical Conditional Probability(SCP) 用來評估詞的內聚性，即這個詞裡面的字元以這個詞序列出現的機率有多大

SCP(gram) = (n-1) (P(gram)^2) / sum( [ P(c1,ci) P(ci+1, cn) for i in range(1,n-1)])

其中n是詞長度。

例如"服貿協議"的SCP，就是"服貿協議"出現的機率P(gram)，去除以以下所有可能分割的機率和之平均：
( P(服)P(貿協議) + P(服貿) P(協議) + P(服貿協) * P(議)) / 3
這裡的P在計算時都直接以詞頻計。

極端的情形下，當詞裡面的所有子序列都只會在出現該詞的情形下出現時，SCP = 1(例如："葡萄")，故對於"渾渾噩噩"之類內聚性高的詞會是很好的指標，高SCP通常代表這很可能是一串常一起出現的詞， 但是由於新詞常常是兩個不常一起出現的詞的組合，例如"大腸花" 所以不能單單用低SCP而判斷一個詞不是新詞。

在此我們只用很低的閾值(scp < 0.005)來去掉明顯有問題的詞，另外由於電腦記憶體不足，只計算了4Gram以下的scp來做篩選。

In [5]:
fourgrams = word_extraction.NGrams(latter_sents, 4, min_count=0,recursive=True)

fr = open('possible_grams','r')
words = [x.strip().decode('utf8') for x in fr if len(x.decode('utf8'))<=5]

word2scp = {word:word_extraction.scp(word,fourgrams.ngrams) for word in words}

word2highscp = {word:scp for word,scp in word2scp.items() if scp > 0.005}

fw = open('high_scp','w')
highscp = sorted(word2highscp,key=lambda x: len(x))
for x in highscp:
    fw.write(x.encode('utf8')+'\n')
fw.close()

## 最後結果

經由上述過程後選出來的詞，之後再由人來判斷其是否為新詞。
又，因為實驗時做到8-gram，此處output出來的詞也會包含它們，
但因為若要做到8-gram記憶體會不足必須分開做，故在上述的過程就沒有demo到8-gram。


In [2]:
for x in open('high_scp','r'):
    print x

綜藝

崇晏

張慶

洋天

莉茲

巫妖

龍田

呀喲

張懸

凍蒜

紋針

黑島

妖王

衛羊

魏揚

參股

召委

冊第

吸塵

豚旗

布朗

戴森

鹿野

塵器

條審

青島

陽花

老李

瓜刀

仁醫

僵屍

卡茸

募款

洪崇

陽餅

紐西

柔性

魏楊

歐羅

濟南

解籤

圾袋

福懋

伊呀

永豐

式爐

瑪莎

仰寧

額寶

察隊

吱谷

八巷

字稿

野百

洋狀

范雲

紫爆

飛帆

林飛

貧富

羅肥

皮糖

飆仔

孬孬

博客

島青

綠衛

吳育

警棍

喬事

鋁梯

逐條

田村

破魔

翁重

餵羊

腸花

便斗

龍斌

官逼

慶忠

熊圖

郝龍

北藝

類電

天佑

西蘭

嗶嗶

壓剪

鋼柱

熱食

方仰

康乃

賢樓

重鈞

群賢

鵝卵

淺藍

乃馨

蔡麗

違停

撐住

迴紋

究員

三熊

麗青

已訂

為廷

建省

確幸

鬼故

博客來

吳育昇

修改權

平等院

條審查

涉及法

文殊菩

停留期

遍地開

盧嘉辰

撐下去

綿羊少

馬卡茸

走不送

光榮碼

洪崇晏

海豚旗

民間版

方公尺

集委員

黑色島

副研究

蔡正元

富差距

內政委

乾女兒

事慣例

二人組

不服從

三重幫

逐條審

吃香蕉

群賢樓

瘋吱谷

路一段

動電源

社科院

假女兒

集體罷

內容涉

罷免案

太陽花

轉移焦

榮碼頭

岸關係

林飛帆

柔性勸

行動電

野草莓

動用警

小便斗

佔領立

裁量權

圓仔射

資源回

林教授

政院核

餘額寶

余昊益

府機關

殊業務

公立醫

凡大帝

丟臉丟

小辣椒

佳龍凍

割闌尾

色正義

宗教信

殭屍ㄟ

黃國昌

大腸花

黨員票

史定位

管機關

數據交

層級化

維持秩

佔主席

件編號

第二類

提神飲

程曉農

馬都督

權利義

蔡丁貴

鐵吱肛

督條例

三條戰

藝節目

森南路

黃復興

物農莊

小海豚

三十秒

礦公司

洋天朝

神預測

金小刀

新台幣

金皮糖

換通信

友爆卦

移焦點

黑島青

口聲聲

理布朗

佑台灣

舉手靜

楊秋興

野協商

教